<a href="https://colab.research.google.com/github/Tixonmavrin/Zindi-Zimnat-Insurance-Recommendation-Challenge/blob/master/Baseline1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Zimnat Insurance Recommendation Challenge
Can you predict which insurance products existing clients will want next?
###Baseline with catboost

### Get data

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt
%matplotlib inline

import os
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, VotingClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn import preprocessing
from sklearn import metrics
import numpy as np
import pandas as pd
from time import time
import pprint
import joblib
from bayes_opt import BayesianOptimization
from sklearn.model_selection import KFold
from xgboost.sklearn import XGBClassifier 
import warnings
warnings.filterwarnings("ignore")

# Classifiers
from catboost import CatBoostClassifier

# Model selection
from sklearn.model_selection import StratifiedKFold

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

# Skopt functions
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer
from time import time

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

# New Section

### New format

In [3]:
X_train = []
X_train_columns = train.columns
c = 0
for v in train.values:
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for i in index:
        c+=1
        for k in range(len(binary)):
            if k == i:
                binary_transformed = list(copy.copy(binary))
                binary_transformed[i] = 0
                X_train.append(list(info) + binary_transformed + [X_train_columns[8+k]] + [c])

X_train = pd.DataFrame(X_train)
X_train.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'ID2']

In [4]:
X_test = []
true_values = []
c = 0
for v in test.values:
    c += 1
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    X_test.append(list(info) + list(binary) + [c])
    for k in test.columns[8:][index]:
        true_values.append(v[0] + ' X ' + k)

X_test = pd.DataFrame(X_test)
X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'ID2']

### Transform data

In [5]:
features_train = []
features_test = []
columns = []

append_features = ['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 
'N2MW', 'AHXO','BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 
'ECY3', 'ID', 'ID2', 'join_date', 'sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code',
'birth_year']
for v in append_features:
    features_train.append(X_train[v].values.reshape(-1, 1))
    features_test.append(X_test[v].values.reshape(-1, 1))
    columns.append(np.array([v]))

y_train = X_train[['product_pred']]

In [6]:
features_train = np.concatenate(features_train, axis=1)
features_test = np.concatenate(features_test, axis=1)
columns = np.concatenate(np.array(columns))

X_train = pd.DataFrame(features_train)
X_train.columns = columns
X_test = pd.DataFrame(features_test)
X_test.columns = columns

### New features

In [7]:
X_train['date1'] = X_train['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
X_train['date2'] = X_train['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
X_train['date3'] = X_train['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
X_train.drop('join_date', axis=1, inplace=True)

X_test['date1'] = X_test['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
X_test['date2'] = X_test['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
X_test['date3'] = X_test['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
X_test.drop('join_date', axis=1, inplace=True)

X_train['date_diff'] = X_train['date3'] - X_train['birth_year']
X_test['date_diff'] = X_test['date3'] - X_test['birth_year']

### Change types

In [8]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
y_train = y_train.fillna(0)

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data = X_train.append(X_test)
for v in ['sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code',]:
    data.loc[:,v] = le.fit_transform(data.loc[:,v])
X_train = data[:X_train.shape[0]]
X_test = data[-X_test.shape[0]:]

In [10]:
le.fit(y_train.iloc[:,0])
y_train = pd.DataFrame(le.transform(y_train.iloc[:,0]))
y_train.columns = ['target']

In [11]:
pd.set_option('display.max_columns', None)
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,1/6/2013,M,U,1991,748L,QZYX,90QI,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,1/8/2019,M,M,1990,1X1H,BP09,56SI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,1/8/2019,M,M,1990,748L,NO3L,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [26]:
train.shape

(29132, 29)

In [12]:
pd.set_option('display.max_columns', None)
X_train.head()

,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,ID,ID2,sex,marital_status,branch_code,occupation_code,occupation_category_code,birth_year,date1,date2,date3,date_diff
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4WKQSBB,1,0,2,0,19,5,1987,1.0,2.0,2019.0,32.0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4WKQSBB,2,0,2,0,19,5,1987,1.0,2.0,2019.0,32.0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,CP5S02H,3,0,2,11,19,5,1981,1.0,6.0,2019.0,38.0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,CP5S02H,4,0,2,11,19,5,1981,1.0,6.0,2019.0,38.0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,2YKDILJ,5,1,6,3,185,1,1991,1.0,6.0,2013.0,22.0


In [13]:
data = X_train.append(X_test)

In [14]:
freq_1 = data['marital_status'].value_counts()
data['marital_status_fe'] = data['marital_status'].map(freq_1)

freq_2 = data['branch_code'].value_counts()
data['branch_code_fe'] = data['branch_code'].map(freq_2)

freq_3 = data['occupation_category_code'].value_counts()
data['occupation_category_code_fe'] = data['occupation_category_code'].map(freq_3)
  

In [15]:
## Squaring the features 

data['marital_status_sq'] = data['marital_status']**2
data['branch_code_sq'] = data['branch_code']**2
data['occupation_code_sq'] = data['occupation_code']**2
data['occupation_category_code_sq'] = data['occupation_category_code']**2
data['date_diff_sq'] = data['date_diff']**2

data['marital_status_cube'] = data['marital_status']**3
data['branch_code_cube'] = data['branch_code']**3
data['occupation_code_cube'] = data['occupation_code']**3
data['occupation_category_code_cube'] = data['occupation_category_code']**3
data['date_diff_cube'] = data['date_diff']**3

data['marital_status_log'] = (data['marital_status']+1).transform(np.log)
data['branch_code_log'] = (data['branch_code']+1).transform(np.log)
data['occupation_code_cube'] = (data['occupation_code']+1).transform(np.log)
data['occupation_category_code_log'] = (data['occupation_category_code']+1).transform(np.log)
data['date_diff_log'] = (data['date_diff']+1).transform(np.log)

data['sex*marital_status'] = data['sex'] * data['marital_status']
data['occupation_code*occupation_cat_code'] = data['occupation_code'] * data['occupation_category_code']
# data = data.drop(['marital_status', 'branch_code', 'occupation_category_code'], axis=1)
# X_train = data[:X_train.shape[0]]
# X_test = data[-X_test.shape[0]:]

In [16]:
data.head()

,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,ID,ID2,sex,marital_status,branch_code,occupation_code,occupation_category_code,birth_year,date1,date2,date3,date_diff,marital_status_fe,branch_code_fe,occupation_category_code_fe,marital_status_sq,branch_code_sq,occupation_code_sq,occupation_category_code_sq,date_diff_sq,marital_status_cube,branch_code_cube,occupation_code_cube,occupation_category_code_cube,date_diff_cube,marital_status_log,branch_code_log,occupation_category_code_log,date_diff_log,sex*marital_status,occupation_code*occupation_cat_code
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4WKQSBB,1,0,2,0,19,5,1987,1.0,2.0,2019.0,32.0,59731,3130,48918,4,0,361,25,1024.0,8,0,2.995732,125,32768.0,1.098612,0.000000,1.791759,3.496508,0,95
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4WKQSBB,2,0,2,0,19,5,1987,1.0,2.0,2019.0,32.0,59731,3130,48918,4,0,361,25,1024.0,8,0,2.995732,125,32768.0,1.098612,0.000000,1.791759,3.496508,0,95
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,CP5S02H,3,0,2,11,19,5,1981,1.0,6.0,2019.0,38.0,59731,8972,48918,4,121,361,25,1444.0,8,1331,2.995732,125,54872.0,1.098612,2.484907,1.791759,3.663562,0,95
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,CP5S02H,4,0,2,11,19,5,1981,1.0,6.0,2019.0,38.0,59731,8972,48918,4,121,361,25,1444.0,8,1331,2.995732,125,54872.0,1.098612,2.484907,1.791759,3.663562,0,95
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,2YKDILJ,5,1,6,3,185,1,1991,1.0,6.0,2013.0,22.0,6905,21481,14842,36,9,34225,1,484.0,216,27,5.225747,1,10648.0,1.945910,1.386294,0.693147,3.135494,6,185


In [17]:
X_train = data[:X_train.shape[0]]
X_test = data[-X_test.shape[0]:]

In [18]:
pbounds = {
    'learning_rate': (0.01, 1.0),
    'min_child_weight': (0, 10),
    'max_depth': (0, 50),
    'max_delta_step': (0, 20),
    'subsample': (0.01, 1.0),
    'colsample_bytree': (0.01, 1.0),
    'colsample_bylevel': (0.01, 1.0),
    'reg_lambda': (1e-9, 1000),
    'reg_alpha': (1e-9, 1.0),
    'gamma': (1e-9, 0.5),
    'min_child_weight': (0, 5),
    'n_estimators': (50, 100)
}
def xgboost_hyper_param(learning_rate,
                        n_estimators,
                        max_depth,
                        subsample,
                        gamma,
                        min_child_weight,
                        max_delta_step,
                       colsample_bytree,
                       colsample_bylevel,
                       reg_lambda,
                       reg_alpha
                       ):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    clf = XGBClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        gamma=gamma,
        min_child_weight=min_child_weight,
        colsample_bytree=colsample_bytree,
        colsample_bylevel=colsample_bylevel,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        max_delta_step=max_delta_step
        )
    return np.mean(cross_val_score(clf, X_train.drop(columns=['ID', 'ID2'], axis=1), y_train, cv=3))
optimizer = BayesianOptimization(
    f=xgboost_hyper_param,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(init_points=3,n_iter=30)

|   iter    |  target   | colsam... | colsam... |   gamma   | learni... | max_de... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

random_search = {'criterion': ['entropy', 'gini'],
               'max_depth': list(np.linspace(10, 1200, 10, dtype = int)) + [None],
               'max_features': ['auto', 'sqrt','log2', None],
               'min_samples_leaf': [4, 6, 8, 12],
               'min_samples_split': [5, 7, 10, 14],
               'n_estimators': list(np.linspace(151, 1200, 10, dtype = int))}

clf = RandomForestClassifier()
model = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 80, 
                               cv = 4, verbose= 5, random_state= 101, n_jobs = -1)
model.fit(X_train.drop(columns=['ID', 'ID2'], axis=1),y_train)

In [ ]:
model.best_estimator_

In [19]:
xgb = XGBClassifier(learning_rate=0.1675, max_depth=10, gamma=1.976, n_estimators=550)
# model_xgb = xgb.fit(X_train.drop(columns=['ID', 'ID2'],axis=1), y_train)
# preds_xgb = model_xgb.predict_proba(X_test.drop(columns=['ID', 'ID2'],axis=1))

In [20]:
rf = RandomForestClassifier(n_estimators=200,max_depth=8,min_samples_split=10)
# model_rf = rf.fit(X_train.drop(columns=['ID', 'ID2'],axis=1), y_train)
# preds_rf = model_rf.predict_proba(X_test.drop(columns=['ID', 'ID2'],axis=1))

In [21]:
gbc = GradientBoostingClassifier(n_estimators=400,learning_rate=0.1,random_state=100,max_features=4 )
# model_gbc = gbc.fit(X_train.drop(columns=['ID', 'ID2'],axis=1), y_train)
# preds_gbc = model_gbc.predict_proba(X_test.drop(columns=['ID', 'ID2'],axis=1))

In [22]:
catb = CatBoostClassifier(depth=7, iterations= 79, learning_rate = 0.24800286296598373, )
# model_catb = catb.fit(X_train.drop(columns=['ID', 'ID2'],axis=1), y_train)
# preds_catb = model_catb.predict_proba(X_test.drop(columns=['ID', 'ID2'],axis=1))

In [23]:
eclf = VotingClassifier(estimators=[('catb', catb)], voting='soft', weights=[1])
model_eclf = eclf.fit(X_train.drop(columns=['ID', 'ID2'],axis=1), y_train)
preds_voting = model_eclf.predict_proba(X_test.drop(columns=['ID', 'ID2'],axis=1))

0:	learn: 1.0172777	total: 207ms	remaining: 16.2s
1:	learn: 0.7988666	total: 342ms	remaining: 13.2s
2:	learn: 0.7198119	total: 471ms	remaining: 11.9s
3:	learn: 0.6674187	total: 604ms	remaining: 11.3s
4:	learn: 0.6268970	total: 734ms	remaining: 10.9s
5:	learn: 0.5968690	total: 856ms	remaining: 10.4s
6:	learn: 0.5758798	total: 996ms	remaining: 10.2s
7:	learn: 0.5549092	total: 1.14s	remaining: 10.1s
8:	learn: 0.5380330	total: 1.27s	remaining: 9.87s
9:	learn: 0.5250719	total: 1.4s	remaining: 9.63s
10:	learn: 0.5134608	total: 1.52s	remaining: 9.4s
11:	learn: 0.5053064	total: 1.72s	remaining: 9.61s
12:	learn: 0.4950017	total: 1.89s	remaining: 9.58s
13:	learn: 0.4880423	total: 2.03s	remaining: 9.41s
14:	learn: 0.4800675	total: 2.17s	remaining: 9.24s
15:	learn: 0.4729832	total: 2.31s	remaining: 9.08s
16:	learn: 0.4674077	total: 2.45s	remaining: 8.94s
17:	learn: 0.4620749	total: 2.58s	remaining: 8.75s
18:	learn: 0.4576790	total: 2.72s	remaining: 8.6s
19:	learn: 0.4537638	total: 2.85s	remaining:

In [24]:
y_test = pd.DataFrame(preds_voting)
y_test.columns = le.inverse_transform(y_test.columns)

In [25]:
y_test

,66FJ,7POT,8NN1,AHXO,BSTQ,ECY3,FM3X,GHYX,GYSR,J9JW,JWFN,JZ9D,K6QO,LJR9,N2MW,P5DA,PYUQ,QBOL,RIBP,RVSZ,SOP4
0,0.000440,0.000060,0.000040,0.000123,0.000150,0.006467,0.000027,0.008732,7.640609e-06,0.000228,0.000195,0.000299,0.956996,0.000115,0.000315,0.000016,0.002415,0.020874,0.001728,0.000015,0.000757
1,0.008403,0.000860,0.000523,0.009791,0.002999,0.289070,0.001356,0.193090,1.037663e-04,0.001158,0.151231,0.002600,0.004823,0.002004,0.009410,0.000145,0.016141,0.285281,0.013246,0.000454,0.007312
2,0.002525,0.000291,0.004132,0.000663,0.000087,0.045781,0.000040,0.003193,4.830024e-05,0.000329,0.006330,0.000698,0.000476,0.000488,0.001962,0.000112,0.004405,0.917839,0.006109,0.000077,0.004415
3,0.000178,0.000057,0.000016,0.000304,0.000157,0.003979,0.000018,0.006110,2.440033e-06,0.000188,0.000202,0.000184,0.977392,0.000193,0.001378,0.000018,0.002625,0.006168,0.000474,0.000013,0.000342
4,0.000078,0.000131,0.000057,0.000073,0.000289,0.000145,0.000020,0.001095,5.131113e-06,0.983842,0.000007,0.002168,0.000012,0.000330,0.000241,0.000041,0.002242,0.005071,0.000070,0.004049,0.000034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.000409,0.000316,0.000042,0.000378,0.001427,0.000357,0.000073,0.010678,1.285468e-05,0.969781,0.000035,0.001545,0.000023,0.000883,0.003891,0.000023,0.001487,0.000229,0.000535,0.007766,0.000110
9996,0.000173,0.000055,0.000089,0.000094,0.000050,0.003079,0.000011,0.007153,5.860114e-06,0.000276,0.000087,0.000316,0.959617,0.000307,0.000315,0.000176,0.005104,0.022557,0.000368,0.000012,0.000156
9997,0.000008,0.000010,0.000002,0.000012,0.000010,0.000016,0.000003,0.000083,6.383215e-07,0.000014,0.000003,0.000015,0.000005,0.000007,0.000024,0.000002,0.000059,0.000045,0.000018,0.999658,0.000008
9998,0.024846,0.002469,0.010932,0.516910,0.002496,0.016817,0.001087,0.003407,4.536063e-04,0.001022,0.000450,0.001470,0.000676,0.002192,0.042629,0.011646,0.044894,0.060683,0.095017,0.126186,0.033717


### Model

### Submit

In [ ]:
answer_mass = []
for i in range(X_test.shape[0]):
    id = X_test['ID'].iloc[i]
    for c in y_test.columns:
        answer_mass.append([id + ' X ' + c, y_test[c].iloc[i]])

df_answer = pd.DataFrame(answer_mass)
df_answer.columns = ['ID X PCODE', 'Label']
for i in range(df_answer.shape[0]):
    if df_answer['ID X PCODE'].iloc[i] in true_values:
        df_answer['Label'].iloc[i] = 1.0

In [ ]:
df_answer.reset_index(drop=True, inplace=True)
df_answer.to_csv('submission49.csv', index=False)

Submit